# Airbnb Data Exploration

Data exploration of Airbnb for London

In [1]:
import pandas as pd
import plotly.express as px
import datetime
import plotly.graph_objects as go

In [2]:
calendar_df = pd.read_csv("../data/calendar.csv")
listing_df = pd.read_csv("../data/listings.csv")

In [3]:
listing_df.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48


In [4]:
calendar_df.head(2)

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00


### Join Calendar and Neighbourhood on listing id

In [5]:
listing_neighbourhood_df = listing_df[['id','neighbourhood_group_cleansed']]
listing_neighbourhood_df = listing_neighbourhood_df.rename(columns={"id": "listing_id"})
merged_df = pd.merge(calendar_df, listing_neighbourhood_df, on='listing_id', how='outer')
merged_df['date'] = pd.to_datetime(merged_df['date'], format="%Y-%m-%d")
merged_df.head(2)

,listing_id,date,available,price,neighbourhood_group_cleansed
0,241032,2016-01-04,t,$85.00,Queen Anne
1,241032,2016-01-05,t,$85.00,Queen Anne


### Clean Price

In [6]:
def clean_string_to_numeric(df, col, character_list):

    for char in character_list:
        df[col] = [x.replace(char,'') for x in df[col]]
    
    return df

def col_to_dtype(df, col, dtype):
    df[col] = df[col].astype(dtype)
    return df

merged_df['price'] = merged_df['price'].astype(str)
merged_df = clean_string_to_numeric(merged_df, 'price', ["$",","])
merged_df = col_to_dtype(merged_df, 'price', float)
merged_df.head()

,listing_id,date,available,price,neighbourhood_group_cleansed
0,241032,2016-01-04,t,85.0,Queen Anne
1,241032,2016-01-05,t,85.0,Queen Anne
2,241032,2016-01-06,f,NaN,Queen Anne
3,241032,2016-01-07,f,NaN,Queen Anne
4,241032,2016-01-08,f,NaN,Queen Anne


In [7]:
calendar_month_avg = merged_df.groupby([merged_df['date'].dt.strftime('%B'), merged_df['neighbourhood_group_cleansed']]).mean()
calendar_month_avg.head()

listing_id       price
date  neighbourhood_group_cleansed                          
April Ballard                       5.180190e+06  120.999350
      Beacon Hill                   5.052427e+06   99.030576
      Capitol Hill                  5.668931e+06  134.181818
      Cascade                       6.089536e+06  157.135593
      Central Area                  5.587679e+06  128.884648

In [8]:
grouped_price_df = calendar_month_avg['price']

In [9]:
unstacked_df = grouped_price_df.unstack(level=1)
unstacked_df

neighbourhood_group_cleansed,Ballard,Beacon Hill,Capitol Hill,Cascade,Central Area,Delridge,Downtown,Interbay,Lake City,Magnolia,Northgate,Other neighborhoods,Queen Anne,Rainier Valley,Seward Park,University District,West Seattle
date,,,,,,,,,,,,,,,,,
April,120.999350,99.030576,134.181818,157.135593,128.884648,89.799889,182.228800,116.575646,92.973451,204.425329,91.769274,121.214977,170.429821,100.901802,105.656733,106.583080,145.114359
August,139.124224,103.394035,142.804368,167.752300,143.500132,96.316934,225.402173,125.200000,101.069565,201.063165,100.167991,135.258759,194.843181,107.465484,121.463768,111.939155,159.073464
December,129.631683,101.333211,133.784395,142.488285,138.567857,92.660020,176.166682,114.500000,90.801760,202.312173,89.121781,129.438850,172.255945,98.605489,119.830688,104.452389,147.817291
February,110.033227,96.401518,125.381330,144.141864,122.093812,81.613109,156.857117,112.874126,92.184322,185.428373,88.005780,113.688757,153.775233,89.511774,97.792434,96.716808,136.937485
January,108.986406,97.777725,123.823421,139.289689,120.513238,78.060000,153.544756,118.723636,88.437179,187.909692,87.301223,110.440733,156.448199,90.091510,100.407538,94.948899,140.519956
July,138.565895,105.394449,144.035177,167.563328,145.537473,94.989714,230.343055,126.659420,101.628116,211.078224,100.652770,134.977329,198.040541,108.731723,119.803419,108.939803,161.445817
June,135.055280,105.828649,141.496378,168.922052,139.640927,92.891257,223.974254,122.118519,102.712753,197.319853,95.683634,129.215547,190.212476,106.635173,110.424658,108.418463,154.083182
March,113.237420,94.018613,129.379577,146.517104,123.147360,87.960275,173.549728,119.141935,91.227027,190.361674,89.282902,114.391564,163.659001,93.054608,97.588752,98.777852,137.891341
May,122.753247,102.737389,135.511436,168.933374,134.314932,91.482906,196.796751,116.682310,96.041793,200.872061,96.053719,124.448971,177.208201,103.639314,106.018085,108.026615,146.612736


In [10]:
unstacked_df['month'] = unstacked_df.index.values

In [11]:
unstacked_df

neighbourhood_group_cleansed,Ballard,Beacon Hill,Capitol Hill,Cascade,Central Area,Delridge,Downtown,Interbay,Lake City,Magnolia,Northgate,Other neighborhoods,Queen Anne,Rainier Valley,Seward Park,University District,West Seattle,month
date,,,,,,,,,,,,,,,,,,
April,120.999350,99.030576,134.181818,157.135593,128.884648,89.799889,182.228800,116.575646,92.973451,204.425329,91.769274,121.214977,170.429821,100.901802,105.656733,106.583080,145.114359,April
August,139.124224,103.394035,142.804368,167.752300,143.500132,96.316934,225.402173,125.200000,101.069565,201.063165,100.167991,135.258759,194.843181,107.465484,121.463768,111.939155,159.073464,August
December,129.631683,101.333211,133.784395,142.488285,138.567857,92.660020,176.166682,114.500000,90.801760,202.312173,89.121781,129.438850,172.255945,98.605489,119.830688,104.452389,147.817291,December
February,110.033227,96.401518,125.381330,144.141864,122.093812,81.613109,156.857117,112.874126,92.184322,185.428373,88.005780,113.688757,153.775233,89.511774,97.792434,96.716808,136.937485,February
January,108.986406,97.777725,123.823421,139.289689,120.513238,78.060000,153.544756,118.723636,88.437179,187.909692,87.301223,110.440733,156.448199,90.091510,100.407538,94.948899,140.519956,January
July,138.565895,105.394449,144.035177,167.563328,145.537473,94.989714,230.343055,126.659420,101.628116,211.078224,100.652770,134.977329,198.040541,108.731723,119.803419,108.939803,161.445817,July
June,135.055280,105.828649,141.496378,168.922052,139.640927,92.891257,223.974254,122.118519,102.712753,197.319853,95.683634,129.215547,190.212476,106.635173,110.424658,108.418463,154.083182,June
March,113.237420,94.018613,129.379577,146.517104,123.147360,87.960275,173.549728,119.141935,91.227027,190.361674,89.282902,114.391564,163.659001,93.054608,97.588752,98.777852,137.891341,March
May,122.753247,102.737389,135.511436,168.933374,134.314932,91.482906,196.796751,116.682310,96.041793,200.872061,96.053719,124.448971,177.208201,103.639314,106.018085,108.026615,146.612736,May


In [12]:


unstacked_df['month_num'] = [datetime.datetime.strptime(x, "%B").month for x in unstacked_df['month']]

In [19]:
monthly_neigh_prices =unstacked_df.sort_values('month_num')
monthly_neigh_prices = monthly_neigh_prices.drop(['month_num','month'], axis=1)
monthly_neigh_prices = monthly_neigh_prices.round(2)
monthly_neigh_prices.head()

neighbourhood_group_cleansed,Ballard,Beacon Hill,Capitol Hill,Cascade,Central Area,Delridge,Downtown,Interbay,Lake City,Magnolia,Northgate,Other neighborhoods,Queen Anne,Rainier Valley,Seward Park,University District,West Seattle
date,,,,,,,,,,,,,,,,,
January,108.99,97.78,123.82,139.29,120.51,78.06,153.54,118.72,88.44,187.91,87.30,110.44,156.45,90.09,100.41,94.95,140.52
February,110.03,96.40,125.38,144.14,122.09,81.61,156.86,112.87,92.18,185.43,88.01,113.69,153.78,89.51,97.79,96.72,136.94
March,113.24,94.02,129.38,146.52,123.15,87.96,173.55,119.14,91.23,190.36,89.28,114.39,163.66,93.05,97.59,98.78,137.89
April,121.00,99.03,134.18,157.14,128.88,89.80,182.23,116.58,92.97,204.43,91.77,121.21,170.43,100.90,105.66,106.58,145.11
May,122.75,102.74,135.51,168.93,134.31,91.48,196.80,116.68,96.04,200.87,96.05,124.45,177.21,103.64,106.02,108.03,146.61


In [26]:

# Create random data with numpy

fig = go.Figure()

# Add traces
for neigh in monthly_neigh_prices.columns:
    fig.add_trace(go.Scatter(x=monthly_neigh_prices.index, y=monthly_neigh_prices[neigh],
                        mode='lines+markers',
                        name=neigh))

fig.update_layout(
    template='plotly_dark',
    autosize=False,
    width=600,
    height=800,
    yaxis=dict(
        title_text="Average Listing Price ($)",
        tickmode="array",
        titlefont=dict(size=14),
    ),
    xaxis=dict(
    title_text="Airbnb eattle Neighbourhood",
    tickmode="array",
    titlefont=dict(size=14),
    ))
fig.show()

In [22]:
import chart_studio
import chart_studio.plotly as py
username = 'jhmarlow' # your username
api_key = 'sW1GO8ncX3JlvRlUGc0M' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

py.plot(fig, filename = 'monthly_neighbourhood_prices_airbnb_seattle', auto_open=True)
